## Spark streaming test

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit_streaming import *
from kafka import KafkaConsumer, TopicPartition

creds, config = read_files()
subreddit = config["subreddit"]
kafka_host = config["kafka_host"]
spark_host = config["spark_host"]
aws_client = creds["aws-client"]
aws_secret = creds["aws-secret"]

print("imported modules")

imported modules


In [2]:
try:
    main()

except KeyboardInterrupt:
    print("stopped spark streaming.")
    
except Exception as e:
    print(e)

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a9e97319-100c-44f5-b1a9-53824b205310;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.

stopped spark streaming.


	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-codec#commons-codec;1.11 in central
	found commons-io#commons-io;2.8.0 in central
	found commons-net#commons-net;3.6 in central
	found commons-collections#commons-collections;3.2.2 in central
	found javax.servlet#javax.servlet-api;3.1.0 in central
	found org.eclipse.jetty#jetty-server;9.4.40.v20210413 in central
	found org.eclipse.jetty#jetty-http;9.4.40.v20210413 in central
	found org.eclipse.jetty#jetty-util;9.4.40.v20210413 in central
	found org.eclipse.jetty#jetty-io;9.4.40.v20210413 in central
	found org.eclipse.jetty#jetty-servlet;9.4.40.v20210413 in central
	found org.eclipse.jetty#jetty-security;9.4.40.v20210413 in central
	found org.eclipse.jetty#jetty-util-ajax;9.4.40.v20210413 in central
	found org.eclipse.jetty#jetty-webapp;9.4.40.v202104

# Check dataset

In [2]:
try:
    spark, sc = init_spark()
    test = spark.read.format("delta").option("header", True).load("s3a://reddit-stevenhurwitt/" + subreddit)
    test_pandas = test.toPandas()
    print("read delta table to df.")

except KeyboardInterrupt:
    print("loading data took too long... cancelled.")

except Exception as e:
    print(e)

/usr/bin/spark-3.2.0-bin-hadoop3.2/conf/spark-env.sh: line 45: export: `=': not a valid identifier
/usr/bin/spark-3.2.0-bin-hadoop3.2/conf/spark-env.sh: line 45: export: `8080': not a valid identifier


:: loading settings :: url = jar:file:/usr/bin/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7a1fb7b2-b262-4342-bbe8-b25313bd3470;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in local-m2-cache
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in local-m2-cache
	found org.apache.kafka#kafka-clients;2.8.0 in local-m2-cache
	found org.lz4#lz4-java;1.7.1 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in local-m2-cache
	found org.spark-project.spark#unused

Java gateway process exited before sending its port number
local variable 'spark' referenced before assignment


In [6]:
test_pandas

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
0,NaN,technology,,t2_hqstdwu0,False,None,0,False,Elon Musk is using a 'dog ate the homework' ex...,r/technology,...,None,/r/technology/comments/uqwpba/elon_musk_is_usi...,all_ads,False,https://markets.businessinsider.com/news/stock...,11994694,1.652710e+09,0,None,False
1,NaN,technology,,t2_7ccf,False,None,0,False,Royal Mail drone fleet takes to the air for re...,r/technology,...,None,/r/technology/comments/uo0r18/royal_mail_drone...,all_ads,False,https://www.edinburghnews.scotsman.com/news/pe...,11965116,1.652361e+09,0,None,False
2,NaN,technology,,t2_7ccf,False,None,0,False,These Nanobots Can Swim Around a Wound and Kil...,r/technology,...,None,/r/technology/comments/uow5as/these_nanobots_c...,all_ads,False,https://www.wired.com/story/these-nanobots-can...,11973872,1.652461e+09,0,None,False
3,NaN,technology,,t2_6utha,False,None,0,False,Did Twitch Violate Texas’ Social Media Law By ...,r/technology,...,None,/r/technology/comments/ur2490/did_twitch_viola...,all_ads,False,https://www.techdirt.com/2022/05/16/did-twitch...,11996188,1.652724e+09,0,None,False
4,NaN,technology,,t2_lz41dbnl,False,None,0,False,"real. fucking. conservative. civil liberties, ...",r/technology,...,None,/r/technology/comments/uo8w92/real_fucking_con...,all_ads,False,https://arstechnica.com/tech-policy/2022/05/te...,11967337,1.652383e+09,0,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,NaN,technology,,t2_83wk7s3r,False,None,0,False,Novel Diamond Semiconductors Operate at Highes...,r/technology,...,None,/r/technology/comments/uqwzkm/novel_diamond_se...,all_ads,False,https://www.ad-na.com/magazine_en/archives/44,11994761,1.652711e+09,0,None,False
74,NaN,technology,,t2_1argepoh,False,None,0,False,Apple iPod creator warns the metaverse will en...,r/technology,...,None,/r/technology/comments/uo6vg8/apple_ipod_creat...,all_ads,False,https://www.bbc.com/news/business-61423268,11966776,1.652378e+09,0,None,False
75,NaN,technology,,t2_a5jwjm1s,False,None,0,False,A colony of blue-green algae can power a compu...,r/technology,...,None,/r/technology/comments/uo75rs/a_colony_of_blue...,all_ads,False,https://interestingengineering.com/blue-green-...,11966852,1.652379e+09,0,None,False
76,NaN,technology,,t2_bf38q9nm,False,None,0,False,"Musk, Twitter CEO spar over bot accounts",r/technology,...,None,/r/technology/comments/ur2j9k/musk_twitter_ceo...,all_ads,False,https://thehill.com/policy/technology/,11996262,1.652726e+09,0,None,False


## Debug main function

In [2]:
try:
    spark = SparkSession.builder.appName("reddit_" + subreddit) \
                .master("spark://{}:7077".format(spark_host)) \
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.0,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1") \
                .config("spark.eventLog.enabled", "true") \
                .config("spark.eventLog.dir", "file:///opt/workspace/events") \
                .config("spark.sql.debug.maxToStringFields", 1000) \
                .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                .enableHiveSupport() \
                .getOrCreate()

    sc = spark.sparkContext
    sc.setLogLevel('WARN')
    # sc._jsc.hadoopConfiguration().set("fs.s3a.awsAccessKeyId", aws_client)
    # sc._jsc.hadoopConfiguration().set("fs.s3a.awsSecretAccessKey", aws_secret)
    # sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")
    print("created spark successfully")

except Exception as e:
    print(e)

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-78141193-5a81-4b03-8895-6d0df63f19d0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.

created spark successfully


In [3]:
stage_df = read_kafka_stream(spark, sc)

22/06/11 21:29:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
try:
        stage_df.writeStream \
                .trigger(processingTime='30 seconds') \
                .outputMode("update") \
                .format("console") \
                .option("truncate", "true") \
                .start() \
                .awaitTermination()

except KeyboardInterrupt:
        print("stopped spark streaming.")

22/06/11 21:30:02 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bafa14ca-1552-475d-970c-aa66b78564a3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/11 21:30:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


stopped spark streaming.


In [7]:
try:
    write_stream(stage_df)

except KeyboardInterrupt:
    print("stopped streaming.")

stopped streaming.


In [6]:
spark.stop()
print("stopped spark successfully.")

stopped spark successfully.


## Test kafka consumer

In [17]:

consumer = KafkaConsumer(
        "reddit_{}".format(subreddit),
        bootstrap_servers=['{}:9092'.format(kafka_host)],
        auto_offset_reset='latest',
        consumer_timeout_ms=1000,
        enable_auto_commit=False,
        auto_commit_interval_ms=1000
    )

In [18]:
try:
    for msg in consumer:
        print(msg)

except KeyboardInterrupt:
    print("stopped consumer.")

stopped consumer.
